# PyTorch Linear Algebra Tutorial

A practical guide to linear algebra with **PyTorch**: tensors, GPU acceleration,
`torch.linalg` ops, autograd with matrix calculus, batching, and sparse tensors.

**Contents**
1. Setup & tensors
2. Device management (CPU/GPU) & dtype
3. Basic ops, broadcasting, and aggregation
4. Norms & distances
5. Solving linear systems (`solve`, `pinv`, `lstsq`)
6. Factorizations (QR / Cholesky / SVD)
7. Eigenvalues/eigenvectors (`eig`, `eigh`)
8. Least squares & regression (closed-form)
9. Autograd with linear algebra
10. Batched linear algebra
11. Sparse tensors (preview)
12. Tips: performance, precision, reproducibility
13. Exercises


## 1) Setup & tensors

In [ ]:
import torch
import math
torch.__version__

In [ ]:
# Basic tensor creation
x = torch.tensor([1., 2., 3.])
X = torch.tensor([[1., 2.], [3., 4.]])
zeros = torch.zeros(2, 3)
ones = torch.ones(2, 3)
eye = torch.eye(3)
x, X, zeros.shape, ones.dtype, eye

## 2) Device management (CPU/GPU) & dtype

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device, torch.get_default_dtype()

In [ ]:
a = torch.arange(6, dtype=torch.float32, device=device).reshape(2, 3)
b = torch.arange(6, dtype=torch.float32, device=device).reshape(3, 2)
c = a @ b
c.device, c.dtype, c

## 3) Basic ops, broadcasting, and aggregation

In [ ]:
u = torch.tensor([1., 2., 3.])
v = torch.tensor([10., 20., 30.])
u + v, u * v, v / u, u.pow(2)

In [ ]:
# Broadcasting example
a = torch.tensor([1., 2., 3.]).reshape(3, 1)
b = torch.tensor([10., 20., 30.]).reshape(1, 3)
a + b  # result is 3x3

In [ ]:
M = torch.arange(1., 10.).reshape(3, 3)
M.sum(), M.mean(), M.std(), M.var(), M.min(), M.max(), M.mean(dim=0), M.mean(dim=1)

## 4) Norms & distances

In [ ]:
v = torch.tensor([3., -4., 12.])
l1 = torch.linalg.vector_norm(v, ord=1)
l2 = torch.linalg.vector_norm(v)
linf = torch.linalg.vector_norm(v, ord=float('inf'))
fro = torch.linalg.matrix_norm(M, ord='fro')
l1, l2, linf, fro

## 5) Solving linear systems (`solve`, `pinv`, `lstsq`)

In [ ]:
A = torch.tensor([[3., 2., -1.], [2., -2., 4.], [-1., 0.5, -1.]])
b = torch.tensor([1., -2., 0.])
x = torch.linalg.solve(A, b)
torch.allclose(A @ x, b)

In [ ]:
# Pseudoinverse-based solve (for rank-deficient/least squares)
pinv = torch.linalg.pinv(A)
x_pinv = pinv @ b
x, x_pinv

In [ ]:
# Least squares (if available in your PyTorch version)
try:
    sol = torch.linalg.lstsq(A, b)
    sol.solution, sol.rank
except Exception as e:
    str(e)  # fallback message

## 6) Factorizations (QR / Cholesky / SVD)

In [ ]:
# QR factorization (economy)
N = torch.tensor([[1., 1.], [1., -1.], [1., 2.]])
Q, R = torch.linalg.qr(N, mode='reduced')
beta = torch.linalg.solve(R, Q.T @ torch.tensor([2., 0., 3.]))
beta, Q.shape, R.shape

In [ ]:
# Cholesky for SPD matrices
S = torch.tensor([[4., 2.], [2., 3.]])
L = torch.linalg.cholesky(S)
rhs = torch.tensor([1., 0.])
y = torch.cholesky_solve(rhs.unsqueeze(1), L)
y.squeeze(), torch.allclose(S @ y.squeeze(), rhs)

In [ ]:
# SVD
A2 = torch.tensor([[1., 0., 0.], [0., 2., 0.], [0., 0., 0.5]]) @ torch.randn(3, 5)
U, Svals, Vh = torch.linalg.svd(A2, full_matrices=False)
rank = (Svals > 1e-12).sum()
pinv_A2 = torch.linalg.pinv(A2)
Svals, rank, pinv_A2.shape

## 7) Eigenvalues/eigenvectors (`eig`, `eigh`)

In [ ]:
eigvals, eigvecs = torch.linalg.eig(A)   # may be complex
eigvals_sym, eigvecs_sym = torch.linalg.eigh(S)  # symmetric
eigvals, eigvals_sym

## 8) Least squares & regression (closed-form)

In [ ]:
torch.manual_seed(0)
X = torch.randn(200, 3)
true_w = torch.tensor([2.0, -1.0, 0.5])
y = X @ true_w + 0.1 * torch.randn(200)

# Closed-form: w = (X^T X)^-1 X^T y  (using pinv for stability)
w_hat = torch.linalg.pinv(X) @ y
w_hat

## 9) Autograd with linear algebra
Differentiate through linear algebra ops to fit models end-to-end.

In [ ]:
w = torch.randn(3, requires_grad=True)
optimizer = torch.optim.SGD([w], lr=0.1)
for _ in range(200):
    optimizer.zero_grad()
    loss = ((X @ w - y) ** 2).mean()
    loss.backward()
    optimizer.step()
w, loss.item()

## 10) Batched linear algebra
PyTorch can solve many small systems in parallel by adding a batch dimension.

In [ ]:
B = 4
A_batch = torch.randn(B, 3, 3)
# Make them well-conditioned by A @ A^T
A_batch = A_batch @ A_batch.transpose(-1, -2)
b_batch = torch.randn(B, 3)
x_batch = torch.linalg.solve(A_batch, b_batch)
torch.allclose(A_batch @ x_batch.unsqueeze(-1), b_batch.unsqueeze(-1), atol=1e-5)

## 11) Sparse tensors (preview)
PyTorch supports sparse layouts for memory efficiency with large sparse matrices.

In [ ]:
indices = torch.tensor([[0, 1, 1], [2, 0, 2]])  # COO indices
values = torch.tensor([3., 4., 5.])
sparse = torch.sparse_coo_tensor(indices, values, size=(2, 3))
dense = torch.randn(3, 2)
prod = torch.sparse.mm(sparse, dense)
sparse, prod

## 12) Tips: performance, precision, reproducibility
- Prefer vectorized/batched ops.
- Move data and models to the **same device** (CPU vs GPU).
- Consider **float64** for numerically sensitive LA tasks: `torch.set_default_dtype(torch.float64)`.
- Use **mixed precision** (`torch.cuda.amp`) for speed on GPUs (careful with stability).
- Set seeds for reproducibility: `torch.manual_seed(0)`.


## 13) Exercises
1. **Solve Ax=b, three RHS**: Factor a random 100×100 SPD matrix (via `cholesky`) and solve for 3 random right-hand sides. Compare to `torch.linalg.solve`.
2. **Least squares**: Create a collinear design matrix; compare `(X^T X)^{-1}X^T y` vs `torch.linalg.lstsq` or `pinv`.
3. **Low-rank SVD**: Build a rank-2 matrix plus small noise; compute SVD and reconstruct the best rank-2 approximation; report Frobenius error.
4. **Autograd + LA**: Fit a linear model with L2 regularization using autograd (ridge regression). Verify that increasing λ shrinks weights.
5. **Batched systems**: Generate a batch of SPD matrices of shape `(B, n, n)` and solve against a batch of RHS vectors in one call. Time vs loop.
